In [34]:
# from sklearn.datasets import fetch_20newsgroups
import pandas as pd

import os
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pandarallel import pandarallel
import multiprocessing
multiprocessing.cpu_count()
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [35]:
neg = pd.read_csv(r'../datasets/negative+CC-900repos.csv')
pos = pd.read_csv(r'../datasets/positive+CC-900repos.csv')

In [36]:
pos['vulnerability'] = 1
neg['vulnerability'] = 0

In [37]:
neg

,github,message,select,diff,vulnerability
0,https://github.com/omniauth/omniauth/commit/0d...,Update README for next dev cycle.,A,diff --git a/README.md b/README.md\nindex fd64...,0
1,https://github.com/omniauth/omniauth/commit/10...,Update tested ruby versions,A,diff --git a/README.md b/README.md\nindex 4a2f...,0
2,https://github.com/omniauth/omniauth/commit/f6...,Remove 2_0-indev from CI tracking,A,diff --git a/.github/workflows/main.yml b/.git...,0
3,https://github.com/omniauth/omniauth/commit/8a...,Merge pull request #1016 from BobbyMcWho/add-t...,A,diff --git a/README.md b/README.md\nindex 3ffb...,0
4,https://github.com/omniauth/omniauth/commit/49...,Merge pull request #1015 from omniauth/make-su...,A,diff --git a/Gemfile b/Gemfile\nindex b0581c01...,0
...,...,...,...,...,...
6342,https://github.com/AntSwordProject/antSword/co...,Merge pull request #252 from yzddmr6/v2.1.x\n\...,A,diff --git a/source/core/asp/index.js b/source...,0
6343,https://github.com/AntSwordProject/antSword/co...,(Enhancement: Module) request 增加垃圾数据填充功能 (thx:...,A,diff --git a/modules/request.js b/modules/requ...,0
6344,https://github.com/AntSwordProject/antSword/co...,release v2.1.8.1,A,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...,0
6345,https://github.com/AntSwordProject/antSword/co...,(Enhance:ShellManager) 新增「自定义数据分割符」配置,A,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...,0


In [ ]:
df = pd.concat([neg[['message','diff','vulnerability']],pos[['message','diff','vulnerability']]],axis=0)
df.fillna('', inplace=True)
# 1是100%的意思
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
id2label={0:'Negative',1:'Positive'}
df.rename(columns={'vulnerability':'labels','diff':'diffs','message':'msgs'},inplace=True)
df = df.replace({"labels": id2label})
df = df.sample(1000)

,msgs,diffs,labels
0,OSClass 2.3.5,diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...,Positive
1,escape html entities to fix xss at the login s...,diff --git a/html/index.php b/html/index.php\n...,Positive
2,0.5.11,diff --git a/client-perl.cgi b/client-perl.cgi...,Negative
3,Fix cleanup of attributes in XML reader\n\nxml...,diff --git a/xmlreader.c b/xmlreader.c\nindex ...,Negative
4,CAMEL-10575: snakeyaml: add an option to filte...,diff --git a/components/camel-snakeyaml/src/ma...,Positive
...,...,...,...
10107,Update README.md,diff --git a/README.md b/README.md\nindex 52f6...,Negative
10108,fix counter error; cluster/stage will check st...,diff --git a/cola/cluster/master.py b/cola/clu...,Negative
10109,Merge pull request #3146 from oliverds/master\...,diff --git a/oc/classes/image.php b/oc/classes...,Negative
10110,firmware: do not show subscription key on firm...,diff --git a/src/opnsense/scripts/firmware/pro...,Negative


In [39]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:




def process_row(row):
    from openai import OpenAI
    import tiktoken
    client = OpenAI(
        api_key="sk-bIHAQMOPc0mMMFKgkvZE9sMyggQvA6zi1SR6oq7uSyZRQpc5",  # 或者 os.environ["OPENAI_API_KEY"]
        base_url="https://sg.uiuiapi.com/v1",  # 注意最好加 /v1,
        # model="gpt-4"
    )
    # Helper method to prompt OpenAI LLM and get response.
    def get_response(user_content):
        response = client.chat.completions.create(
        model="gpt-4",  # 根据代理方文档修改
        messages=[
            {"role": "user", "content": user_content}
        ]
        )
        print(response.choices[0].message.content)
        
        return response.choices[0].message.content

    # text = "\'How can I change my pin?\'"
    # examples = get_examples(examples_pool)
    # df = pd.read_csv('dataset.csv')
    # Helper to format the k-shot prompt with:
    # - prefix
    # - 1 example from each class
    # - target text for classification
    def num_tokens_from_messages(messages, model="gpt-4"):
        """Returns the number of tokens used by a list of messages."""
        try:
            encoding = tiktoken.encoding_for_model(model)
        except KeyError:
            encoding = tiktoken.get_encoding("cl100k_base")
        if model == "gpt-4-0301":  # note: future models may deviate from this
            num_tokens = 0
            for message in messages:
                num_tokens += (
                    4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
                )
                for key, value in message.items():
                    num_tokens += len(encoding.encode(value))
                    if key == "name":  # if there's a name, the role is omitted
                        num_tokens += -1  # role is always required and always 1 token
            num_tokens += 2  # every reply is primed with <im_start>assistant
            return num_tokens
        else:
            raise NotImplementedError(
                f"""num_tokens_from_messages() is not presently implemented for model {model}.
    See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
            )

    def generate_promot(row):
        base_promot = """
        Please act as a security patch identificator,
        categorize patch into two categories: Negative, Positive.
        The Negative category corresponds to patches which includes insecure code changes.
        The Positive category corresponds to patches which includes secure code changes.
        I will provide you with the commit message and code diff for a commit patch,
        and you need to give me the category label for this commit patch.
        Please avoid any explanations and only provide the category label.
        """

        commit_message_prompt = f"commit message: {row['msgs']}\n"
        commit_codediff_prompt = f"code diff: {row['diffs']}\n"

        prompt = base_promot + commit_message_prompt + commit_codediff_prompt

        messages = [{"role": "user", "content": prompt}]

        l, r = 0, len(commit_codediff_prompt) - 1
        while l <= r:
            mid = (l + r) // 2
            prompt = base_promot + commit_message_prompt + commit_codediff_prompt[:mid]
            messages = [{"role": "user", "content": prompt}]

            ok = num_tokens_from_messages(messages) < 4096
            if ok:
                l = mid + 1
            else:
                r = mid - 1
        prompt = base_promot + commit_message_prompt + commit_codediff_prompt[:r]

        return prompt
    user_content = generate_promot(row)


    return get_response(user_content)

val_df['result'] = val_df.parallel_apply(process_row, axis=1)


In [41]:
val_df['result']

7762    Negative
7566    Positive
7606    Positive
5272    Positive
7537    Positive
          ...   
6112    Positive
274     Positive
480     Positive
5688    Positive
6518    Positive
Name: result, Length: 1517, dtype: object

In [42]:
val_df['labels']

7762    Positive
7566    Negative
7606    Positive
5272    Negative
7537    Negative
          ...   
6112    Positive
274     Positive
480     Negative
5688    Positive
6518    Negative
Name: labels, Length: 1517, dtype: object

In [43]:
val_df.to_csv('900repo_result.csv',encoding='utf_8_sig')

In [44]:
# # 仅保留 result 为 'positive' 或 'negative' 的行
# val_df = val_df[val_df['result'].isin(['positive', 'negative'])].copy()

# # 打印对齐检查（行数应一致）
# print(f"After filtering, len(result)={len(val_df['result'])}, len(labels)={len(val_df['labels'])}")

# # 比较结果是否一致
# comparison = val_df['result'] == val_df['labels']

# # 查看不一致的情况
# mismatch_df = val_df[~comparison]

# print(f"共有 {len(mismatch_df)} 行不匹配。")
# display(mismatch_df.head())

In [45]:
val_df['result'] = val_df['result'].replace({'(Positive)': 'Positive','The commit is categorized as Positive.': 'Positive', 'Positive': 'Positive','Negative':'Negative'})

In [46]:
val_df['result'].value_counts()

result
Positive    1393
Negative     124
Name: count, dtype: int64

In [47]:
# val_df['result']

In [48]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(val_df['labels'],val_df['result'],digits=4))

              precision    recall  f1-score   support

    Negative     0.6371    0.0862    0.1519       916
    Positive     0.3991    0.9251    0.5577       601

    accuracy                         0.4186      1517
   macro avg     0.5181    0.5057    0.3548      1517
weighted avg     0.5428    0.4186    0.3127      1517



In [ ]:
1

1